### Lets implement the sub question 5


In [172]:
import pandas as pd
import re
from statsmodels.distributions.empirical_distribution import ECDF
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bootstrap


In [ ]:
channels_df = pd.read_csv(
    "../data/df_channels_en.tsv.gz", compression="infer", sep="\t"
)


In [ ]:
metadatas_df = pd.read_feather(
    "../data/yt_metadata_helper.feather",
    columns=[
        "categories",
        "upload_date",
        "duration",
        "like_count",
        "dislike_count",
        "view_count",
        "channel_id",
    ],
)


In [143]:
channels_df.head()


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [132]:
metadatas_df.head()


,categories,upload_date,duration,like_count,dislike_count,view_count,channel_id
0,Film & Animation,2016-09-28,1159,8.0,1.0,1057.0,UCzWrhkg9eK5I8Bm3HfV-unA
1,Film & Animation,2016-09-28,2681,23.0,1.0,12894.0,UCzWrhkg9eK5I8Bm3HfV-unA
2,Film & Animation,2016-09-28,1394,1607.0,779.0,1800602.0,UCzWrhkg9eK5I8Bm3HfV-unA
3,Film & Animation,2016-09-28,5064,227.0,24.0,57640.0,UCzWrhkg9eK5I8Bm3HfV-unA
4,Film & Animation,2016-09-28,3554,105.0,13.0,86368.0,UCzWrhkg9eK5I8Bm3HfV-unA


In [ ]:
metadatas_df.shape


Use the metadatas to notice if channels changed their content (category) sometimes.


In [ ]:
INTERESTING_CAT = ["Gaming", "People & Blogs", "Comedy"]
INTERESTING_CAT_IDS = channels_df[channels_df.category_cc.isin(INTERESTING_CAT)].channel


In [ ]:
# select the channels as main topic "Gaming" and which are present in the channels df
comedy_ids = channels_df[channels_df.category_cc == "Gaming"].channel
# select these channels in the metadatas
metadatas_gaming = metadatas_df[metadatas_df.channel_id.isin(comedy_ids)]
metadatas_gaming.categories.value_counts().plot(kind="bar")
plt.title("Number of videos per categories for channels with Gaming topic")
plt.xlabel("Categories")
plt.ylabel("Number of videos")


In [ ]:
# select the channels as main topic "People & Blogs" and which are present in the channels df
comedy_ids = channels_df[channels_df.category_cc == "People & Blogs"].channel
# select these channels in the metadatas
metadatas_people_blogs = metadatas_df[metadatas_df.channel_id.isin(comedy_ids)]
metadatas_people_blogs.categories.value_counts().plot(kind="bar")
plt.title("Number of videos per categories for channels with People and Blogs topic")
plt.xlabel("Categories")
plt.ylabel("Number of videos")


In [ ]:
# select the channels as main topic "Comedy" and which are present in the channels df
comedy_ids = channels_df[channels_df.category_cc == "Comedy"].channel
# select these channels in the metadatas
metadatas_comedy = metadatas_df[metadatas_df.channel_id.isin(comedy_ids)]
metadatas_comedy.categories.value_counts().plot(kind="bar")
plt.title("Number of videos per categories for channels with Comedy topic")
plt.xlabel("Categories")
plt.ylabel("Number of videos")


Find channels with unique topic


In [156]:
comedy_channels_with_categories = metadatas_comedy.groupby("channel_id").apply(
    lambda group: group.categories.unique()
)
unique_topic_comedy = comedy_channels_with_categories.loc[
    comedy_channels_with_categories.str.len() == 1
]
non_unique_topic_comedy = comedy_channels_with_categories.loc[
    comedy_channels_with_categories.str.len() > 1
]
print("nb of channels with unique topic Comedy : ", unique_topic_comedy.shape)
print("nb of channels without unique topic Comedy : ", non_unique_topic_comedy.shape)


nb of channels with unique topic Comedy :  (536,)
nb of channels without unique topic Comedy :  (3231,)


In [157]:
blogs_channels_with_categories = metadatas_people_blogs.groupby("channel_id").apply(
    lambda group: group.categories.unique()
)
unique_topic_blog = blogs_channels_with_categories.loc[
    blogs_channels_with_categories.str.len() == 1
]
non_unique_topic_blog = blogs_channels_with_categories.loc[
    blogs_channels_with_categories.str.len() > 1
]
print("nb of channels with unique topic People & Blogs : ", unique_topic_blog.shape)
print(
    "nb of channels without unique topic People & Blogs : ", non_unique_topic_blog.shape
)


nb of channels with unique topic People & Blogs :  (8723,)
nb of channels without unique topic People & Blogs :  (9690,)


In [158]:
gaming_channels_with_categories = metadatas_gaming.groupby("channel_id").apply(
    lambda group: group.categories.unique()
)
unique_topic_gaming = gaming_channels_with_categories.loc[
    gaming_channels_with_categories.str.len() == 1
]
non_unique_topic_gaming = gaming_channels_with_categories.loc[
    gaming_channels_with_categories.str.len() > 1
]
print("nb of channels with unique topic Gaming : ", unique_topic_gaming.shape)
print("nb of channels without unique topic Gaming : ", non_unique_topic_gaming.shape)


nb of channels with unique topic Gaming :  (5952,)
nb of channels without unique topic Gaming :  (14191,)


In [159]:
unique_topic_comedy_channels_descript = channels_df[
    channels_df.channel.isin(unique_topic_comedy.index.values)
]
non_unique_topic_comedy_channels_descript = channels_df[
    channels_df.channel.isin(non_unique_topic_comedy.index.values)
]


In [160]:
unique_topic_blogs_channels_descript = channels_df[
    channels_df.channel.isin(unique_topic_blog.index.values)
]
non_unique_topic_blogs_channels_descript = channels_df[
    channels_df.channel.isin(non_unique_topic_blog.index.values)
]


In [161]:
unique_topic_gaming_channels_descript = channels_df[
    channels_df.channel.isin(unique_topic_gaming.index.values)
]
non_unique_topic_gaming_channels_descript = channels_df[
    channels_df.channel.isin(non_unique_topic_gaming.index.values)
]


### Now we have the dataset to check the different between unique and non unique categories channels


In [200]:
def get_confidence_interval(data):
    rng = np.random.default_rng()
    return bootstrap(
        (data,),
        np.mean,
        confidence_level=0.95,
        random_state=rng,
    ).confidence_interval


In [201]:
conf1 = get_confidence_interval(unique_topic_comedy_channels_descript.subscribers_cc)
print(
    "95% confidence interval of number of subs for unique Comedy channels [{:_}, {:_}]".format(
        int(conf1[0]), int(conf1[1])
    )
)
conf2 = get_confidence_interval(
    non_unique_topic_comedy_channels_descript.subscribers_cc
)
print(
    "95% confidence interval of number of subs for non unique Comedy channels [{:_}, {:_}]".format(
        int(conf2[0]), int(conf2[1])
    )
)


95% confidence interval of number of subs for unique Comedy channels [320_024, 504_812]
95% confidence interval of number of subs for non unique Comedy channels [394_339, 495_778]


In [202]:
conf1 = get_confidence_interval(unique_topic_blogs_channels_descript.subscribers_cc)
print(
    "95% confidence interval of number of subs for unique Blogs channels [{:_}, {:_}]".format(
        int(conf1[0]), int(conf1[1])
    )
)
conf2 = get_confidence_interval(non_unique_topic_blogs_channels_descript.subscribers_cc)
print(
    "95% confidence interval of number of subs for non unique Blogs channels [{:_}, {:_}]".format(
        int(conf2[0]), int(conf2[1])
    )
)


95% confidence interval of number of subs for unique Blogs channels [111_597, 129_930]
95% confidence interval of number of subs for non unique Blogs channels [175_958, 204_530]


In [203]:
conf1 = get_confidence_interval(unique_topic_gaming_channels_descript.subscribers_cc)
print(
    "95% confidence interval of number of subs for unique Gaming channels [{:_}, {:_}]".format(
        int(conf1[0]), int(conf1[1])
    )
)
conf2 = get_confidence_interval(
    non_unique_topic_gaming_channels_descript.subscribers_cc
)
print(
    "95% confidence interval of number of subs for non unique Gaming channels [{:_}, {:_}]".format(
        int(conf2[0]), int(conf2[1])
    )
)


95% confidence interval of number of subs for unique Gaming channels [158_822, 185_439]
95% confidence interval of number of subs for non unique Gaming channels [199_496, 242_447]
